# Baseline Indicator Update Process
This notebook is designed to update the CLSS Baseline Indicators in the Dev, Test, and Demo feature services using the data from the `CLSS Baseline Indicators Table.xlsx` file. Additionally, it updates the `SystemTemplate.json` file in the GitHub repository with the updated indicators.

#### Prerequisites
Before running this notebook, please ensure the following:
1. **Upload Required Files**: Upload the `CLSS Baseline Indicators Table.xlsx` file to the `/home` directory in ArcGIS Online/Notebook.
2. **GitHub Personal Access Token**: Generate a GitHub Personal Access Token with the necessary permissions (e.g., `repo` scope) and update the token in the notebook where indicated.
3. **Configuration File**: Ensure that a `config.json` file is present in the `/home` directory. This file should contain your ArcGIS Online (AGO) username and password, as well as the github token in the following format:
     ```json
     {
       "AGO_username": "your_username",
       "AGO_password": "your_password",
       "GH_token": "your token"
     }
     ```

#### Steps
Follow the steps outlined above to ensure all prerequisites are met. Then, run the cells in the notebook sequentially to update the baseline indicators in the feature services and the GitHub repository.
1. **Load and Validate Files**: The notebook will check for the presence of the `CLSS Baseline Indicators Table.xlsx` file and the `config.json` file in the `/home` directory.
2. **Authenticate with ArcGIS Online**: The notebook will authenticate with ArcGIS Online using the credentials provided in the `config.json` file.
3. **Select Feature Service**: Use the dropdown menu to select the target CLSS feature service for updating the baseline indicators.
4. **Update Feature Service**: The notebook will update the selected feature service with the new baseline indicators.
5. **Update GitHub Repository**: The notebook will update the `SystemTemplate.json` file in the GitHub repository with the new baseline indicators.

#### Notes
- GitHub dependencies are not included in the core packages of ArcGIS Notebooks, so they are commented out. The notebook uses the `requests` module to interact with the GitHub API.
- Ensure that the `CLSS Baseline Indicators Table.xlsx` file is correctly formatted and contains the necessary data for updating the feature services and the GitHub repository.

#### Links
- [DevOps Story #1779](https://dev.azure.com/ghinternational/GHIS/_workitems/edit/1779)
- [GitHub Repository: clss-deployment](https://github.com/GH-International/clss-deployment/blob/main/assets/SystemTemplate.json)


In [ ]:
# Standard library imports
import os
import json
import base64

# Third-party library imports
import requests
import pandas as pd
from arcgis.gis import GIS
import ipywidgets as widgets

In [ ]:

# Check if /home folder exists, if not create it
home_dir = 'home'
if not os.path.exists(home_dir):
    os.makedirs(home_dir)

# Get username and password from config file
config_path = f'{home_dir}/config.json'
if not os.path.isfile(config_path):
    raise FileNotFoundError(f"Config file '{config_path}' does not exist. Please upload 'config.json' to the home directory.")

with open(config_path) as config_file:
    config = json.load(config_file)
    
GH_token = config.get('github-token')

# Local path to CLSS Baseline Indicators Table
file_path = f'{home_dir}/CLSS Baseline Indicators Table.xlsx'

# Check if the file exists in the home directory
if not os.path.isfile(file_path):
    raise FileNotFoundError(f"File '{file_path}' does not exist. Please upload 'CLSS Baseline Indicators Table.xlsx' to the home directory.")

# Authenticate with ArcGIS Online
gis = GIS('home')

# This section not needed if running in ArcGIS ONline
# if not AGO_username or not AGO_password:
#     raise ValueError("AGO_username and AGO_password must be provided in the config.json file.")
# AGO_username = config.get('AGO_username')
# AGO_password = config.get('AGO_password')
# try:
#     gis = GIS("https://ghis.maps.arcgis.com/", AGO_username, AGO_password)
# except Exception as e:
#     raise ConnectionError(f"Failed to authenticate with ArcGIS Online: {e}")

# These are the CLSS ArcGIS online Service Endpoints
clssServices = [
    ('None', 'None'),  # Example ID for Dev
    ('CLSS EE TEST', '980b62dffcb94c10bff227e9a8a43fce'),
    ('CLSS Dev', '4cec9a93384543e0a676e3ad892362bb'),
    ('CLSS Test', 'f726c24e77d5442c9a9f456eec62ae5d'),
    ('CLSS Demo', 'c5aea531e01d49358b73123e334b4c0a')
]

In [51]:
# Read the Excel file into a DataFrame
def loadFile(file_path):
    return pd.read_excel(file_path)


In [52]:
baseline_template = loadFile(file_path)

# Generate the JSON structure
content_json = []
for lifeline_name, lifeline_group in baseline_template.groupby("Lifeline"):
    lifeline_entry = {
        # "id": generate_uuid(),
        "name": lifeline_name,
        "title": lifeline_name,
        "componentTemplates": []
    }
    
    for component_name, component_group in lifeline_group.groupby("Component"):
        component_entry = {
            # "id": generate_uuid(),
            "name": component_name,
            "title": component_name,
            "indicators": []
        }
        
        for _, row in component_group.iterrows():
            indicator_entry = {
                # "id": generate_uuid(),
                "name": row["Indicator"],
                "templateName": "CLSS System Baseline",
                "isNew": False,
                "isBeingEdited": False,
                "lifelineName": lifeline_name,
                "componentName": component_name,
                # "componentId": component_entry["id"],
                # "templateId": generate_uuid(),
                "weight": 1
            }
            component_entry["indicators"].append(indicator_entry)
        
        lifeline_entry["componentTemplates"].append(component_entry)
    
    content_json.append(lifeline_entry)

# Convert to JSON format and print or save
json_output = json.dumps(content_json, indent=4)
# print(json_output)


In [ ]:
# Function to handle the feature services selection
def on_service_selected(change):
    global target_template_table_df,target_template_table
    selected_service_id = change['new']
    selected_feature_service = gis.content.get(selected_service_id)
    target_template_table = selected_feature_service.tables[9]
    print(f"Destination FS: {selected_feature_service.title}")
    target_template_table_df = target_template_table.query(where="Name = 'CLSS System Baseline'").sdf
    # print(target_template_table_df)
    return target_template_table_df, target_template_table

# Create a dropdown widget for selecting the CLSS feature service
service_choice = widgets.Dropdown(
    options=clssServices,
    description='Select FS:',
    disabled=False,
)

# Display the dropdown widget & attach the handler
display(service_choice)
service_choice.observe(on_service_selected, names='value')


## Update CLSS Feature Service: Template Table

In [ ]:
# Create the 'update features' JSON object
update_features_json = [{
    "attributes": {
        "OBJECTID": target_template_table_df.at[0, "OBJECTID"],
        "Content": json.dumps(content_json, indent=4)
    }
}]

# Apply the edits to the feature service using the pre-defined update_features_json
response = target_template_table.edit_features(updates=update_features_json)

print("Response", response)

Response {'addResults': [], 'updateResults': [{'objectId': 23, 'uniqueId': 23, 'globalId': None, 'success': True}], 'deleteResults': []}


## Update Github Repo

In [ ]:
### update github repo with requests module

# GitHub repository details
owner = 'GH-International'
repo = 'clss-deployment'
branch = 'main'  # or the branch you want to update
file_path = 'assets/SystemTemplate-test.json'
commit_message = 'Update from SharePoint Template via api'

# Encode the content to base64
encoded_content = base64.b64encode(baseline_template_json.encode()).decode()

# GitHub API URL to get the file's SHA
url = f'https://api.github.com/repos/{owner}/{repo}/contents/{file_path}'

# Headers for authentication
headers = {
    'Authorization': f'token {GH_token}',
    'Accept': 'application/vnd.github.v3+json'
}

# Get the file's SHA
response = requests.get(url, headers=headers)
response_data = response.json()
file_sha = response_data['sha']

# Prepare the data for the update request
data = {
    'message': commit_message,
    'content': encoded_content,
    'sha': file_sha,
    'branch': branch
}

# Make the PUT request to update the file
response = requests.put(url, headers=headers, data=json.dumps(data))

# Check the response
if response.status_code == 200:
    print('File updated successfully.')
else:
    print(f'Failed to update file: {response.status_code}')
    print(response.json())

File updated successfully.
